### This is the pre-training of text-aware masked color model for full palette generation
- It takes 10mins on GPU (Tesla T4 * 1) for one time training with early stop (patience=30)
- For a quick start, do Pretraining with pre-created color corpus files and text embedding files
    - To create color and text data for training, please check preprocess.ipynb

In [1]:
import tensorflow as tf

print(len(tf.config.experimental.list_physical_devices('GPU')))

2023-09-04 05:31:20.996437: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0


1


2023-09-04 05:31:22.732604: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2023-09-04 05:31:22.740454: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-09-04 05:31:22.741027: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.75GiB deviceMemoryBandwidth: 298.08GiB/s
2023-09-04 05:31:22.741094: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.11.0
2023-09-04 05:31:22.745369: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.11
2023-09-04 05:31:22.747363: I tensorflow/stream_executor/platform/default/d

In [2]:
import pandas as pd
from collections import defaultdict  # For word frequency
import os
import math
import random
import ast
from datetime import datetime
from collections import Counter
import numpy as np

import sys
sys.path.append('../src')

import full_palette_generation.text_color_model.color_model as Model
from full_palette_generation.text_color_model.input_data_generator import DataGenerator
from full_palette_generation.text_color_model.model_config import Config

In [3]:
def calculate_pretrain_task_accuracy(mlm_predict, batch_mlm_mask, origin_x):
    
    batch_mlm_mask = tf.cast(batch_mlm_mask, dtype=tf.int32)
    index = tf.where(batch_mlm_mask == 1)
    x_predict = tf.math.argmax(mlm_predict, axis=-1)
    x_predict = tf.gather_nd(x_predict, index)
    x_real = tf.gather_nd(origin_x, index)
    mlm_accuracy = tf.keras.metrics.Accuracy()
    mlm_accuracy.update_state(x_predict, x_real)
    mlm_accuracy = mlm_accuracy.result().numpy()

    return mlm_accuracy

In [7]:
# pretrain

for n in range(1):
    # physical_devices = tf.config.experimental.list_physical_devices('CPU')
    physical_devices = tf.config.experimental.list_physical_devices('GPU')
    assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

    model = Model.Text2Palettes(Config)
    optimizer = tf.keras.optimizers.Adam(Config['Learning_Rate'])
    loss_fn = Model.Text2Palettes_Loss()
    dataset = DataGenerator(Config)

    # create the data for validation and test
    PROJECT_PATH = Config['project_path']
    representation = Config['representation']
    text_model = Config['text_model']
    emb_file = Config['emb_file']

    Config_val = Config.copy()
    Config_val['Corpus_File_Path'] = os.path.join(PROJECT_PATH, f'Data_color/color_corpus_{representation}_val.txt')
    Config_val['Text_Input_File_Path'] = os.path.join(PROJECT_PATH, f'Data_text/text_input_val.txt')
    Config_val['Text_Input_Emb_File_Path'] = os.path.join(PROJECT_PATH, f'Data_text/{emb_file}/text_input_emb{text_model}_val.txt')

    Config_test = Config.copy()
    Config_test['Corpus_File_Path'] = os.path.join(PROJECT_PATH, f'Data_color/color_corpus_{representation}_test.txt')
    Config_test['Text_Input_File_Path'] = os.path.join(PROJECT_PATH, f'Data_text/text_input_test.txt')
    Config_test['Text_Input_Emb_File_Path'] = os.path.join(PROJECT_PATH, f'Data_text/{emb_file}/text_input_emb{text_model}_test.txt')

    dataset_val = DataGenerator(Config_val)
    dataset_test = DataGenerator(Config_test)


    patience = 50 # baseline:10
    best = math.inf
    wait = 0

    Config['Saved_Weight'] = os.path.join(PROJECT_PATH, f'Saved_Weight{text_model}_{Config["Embedding_Size"]}d_{representation}_{Config["Mask_Rate"]}_{Config["Mask_Token_Rate"]}_v{n}')
    checkpoint = tf.train.Checkpoint(model=model)
    checkpoint.restore(tf.train.latest_checkpoint(Config['Saved_Weight']))
    manager = tf.train.CheckpointManager(checkpoint, directory=Config['Saved_Weight'], max_to_keep=5)
    log_dir = os.path.join(Config['Log_Dir'], datetime.now().strftime("%Y-%m-%d"))
    writer = tf.summary.create_file_writer(log_dir)
    
    EPOCH = 2000 # 2000 when training
    for epoch in range(EPOCH):
    #     print(f'dataset length: {len(dataset)}')
        for step in range(len(dataset)):
            batch_x, batch_mlm_mask, batch_mcc_mask, origin_x, batch_segment, batch_padding_mask, batch_text_input_embed = dataset[step]

            with tf.GradientTape() as t:
                mlm_predict, sequence_output = model((batch_x, batch_mlm_mask, batch_segment, batch_text_input_embed), training=True)

                mlm_loss = loss_fn((mlm_predict, batch_mlm_mask, origin_x))
                mlm_loss = tf.reduce_mean(mlm_loss)

                loss = mlm_loss

            gradients = t.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))

            mlm_acc = calculate_pretrain_task_accuracy(mlm_predict, batch_mlm_mask, origin_x)

            if step == len(dataset) - 1 and epoch % 10 == 0:
                print(
                    'Epoch {}, step {}, loss {:.4f}, mlm_loss {:.4f}, mlm_acc {:.4f}'.format(
                        epoch, step, loss.numpy(),
                        mlm_loss.numpy(),
                        mlm_acc,
                        ))

        for val_step in range(len(dataset_val)):
            val_batch_x, val_batch_mlm_mask, val_batch_mcc_mask, val_origin_x, val_batch_segment, val_batch_padding_mask, val_batch_text_input_embed = dataset_val[val_step]

            val_mlm_predict, val_sequence_output = model((val_batch_x, val_batch_mlm_mask, val_batch_segment, val_batch_text_input_embed), training=False)

            val_mlm_loss = loss_fn((val_mlm_predict, val_batch_mlm_mask, val_origin_x))
            val_mlm_loss = tf.reduce_mean(val_mlm_loss)
            
            val_mlm_acc = calculate_pretrain_task_accuracy(val_mlm_predict, val_batch_mlm_mask, val_origin_x)

            val_loss = val_mlm_loss

            if val_step == len(dataset_val) - 1 and epoch % 10 == 0:
                print(
                    'Val: Epoch {}, step {}, loss {:.4f}, mlm_loss {:.4f}, mlm_acc {:.4f}'.format(
                        epoch, val_step, val_loss.numpy(),
                        val_mlm_loss.numpy(),
                        val_mlm_acc,
                        ))
        
        path = manager.save(checkpoint_number=epoch)

        # early stopping
        wait += 1
        if val_loss < best:
            best = val_loss
            wait = 0
        if wait >= patience:
            break
            
    for test_step in range(len(dataset_test)):
        test_batch_x, test_batch_mlm_mask, test_batch_mcc_mask, test_origin_x, test_batch_segment, test_batch_padding_mask, test_batch_text_input_embed = dataset_test[test_step]
            
        test_mlm_predict, test_sequence_output = model((test_batch_x, test_batch_mlm_mask, test_batch_segment, test_batch_text_input_embed), training=False)

        test_mlm_loss = loss_fn((test_mlm_predict, test_batch_mlm_mask, test_origin_x))
        test_mlm_loss = tf.reduce_mean(test_mlm_loss)

        test_mlm_acc = calculate_pretrain_task_accuracy(test_mlm_predict, test_batch_mlm_mask, test_origin_x)

        test_loss = test_mlm_loss
        
        if test_step == len(dataset_test) - 1:
            print(
                'Test: Epoch {}, step {}, loss {:.4f}, mlm_loss {:.4f}, mlm_acc {:.4f}'.format(
                    epoch, test_step, test_loss.numpy(),
                    test_mlm_loss.numpy(),
                    test_mlm_acc,
                    ))
    
    # model.save(f'../data/trained_model/t2p_ca1_mca1_stop{patience}_lr{Config["Learning_Rate"]}_dr{transformer_dropout_rate}_{ca_dropout_rate}_{text_model}_{Config["Embedding_Size"]}d_{representation}_{Config["Mask_Rate"]}_{Config["Mask_Token_Rate"]}_{n}')



If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Epoch 0, step 7,